# ACCIDENTS AND VICTIMS BY TIMEFRAMES

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import sidetable

In [2]:
#Import dataframe and storing raw data as copy.
acc_raw = pd.read_csv('../datasets/2019_accidents_gu_bcn.csv')
acc = acc_raw.copy()

In [3]:
acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10027 entries, 0 to 10026
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Numero_expedient           10027 non-null  object 
 1   Codi_districte             10027 non-null  int64  
 2   Nom_districte              10027 non-null  object 
 3   Codi_barri                 10027 non-null  int64  
 4   Nom_barri                  10027 non-null  object 
 5   Codi_carrer                10027 non-null  int64  
 6   Nom_carrer                 10027 non-null  object 
 7   Num_postal_caption         10027 non-null  object 
 8   Descripcio_dia_setmana     10027 non-null  object 
 9   Dia_setmana                10027 non-null  object 
 10  Descripcio_tipus_dia       10027 non-null  object 
 11  NK_Any                     10027 non-null  int64  
 12  Mes_any                    10027 non-null  int64  
 13  Nom_mes                    10027 non-null  obj

In [4]:
# the type of day, shown in 'Descripcio_tipus_dia' is not filled correctly, ass all days are classified as labour.
print(acc['Descripcio_tipus_dia'].value_counts())
# therefore we will drop this column
acc.drop(columns='Descripcio_tipus_dia', inplace=True)

Laboral    10027
Name: Descripcio_tipus_dia, dtype: int64


In [5]:
#creating a new column for the type of day (if weekday or weekend/holiday). 1st set all as weekday
acc['Type_day'] = 'Weekday'

#setting the weekends as Weekend/Holiday
acc.loc[acc['Dia_setmana'].isin(['Ds','Dg']),'Type_day'] = 'Weekend/Holiday'

# Creating another new column with the complete date of the accident as a string
acc['Date'] = acc['Dia_mes'].astype(str)+'/'+acc['Mes_any'].astype(str)+'/'+acc['NK_Any'].astype(str)

# Importing the official Holidays of Barcelona during 2019, as a list.
holidays = pd.read_csv('../datasets/WeekendsHolidays.csv',delimiter=';')['Date'].astype(str).to_list()

# setting the holidays as Weekend/Holiday
acc.loc[acc['Date'].isin(holidays),'Type_day'] = 'Weekend/Holiday'

In [6]:
acc_clean = acc.drop(columns=['Codi_districte', 'Codi_barri', 'Nom_carrer', 'Num_postal_caption', 
                              'Descripcio_dia_setmana', 'Descripcio_causa_vianant', 'Coordenada_UTM_X', 
                              'Coordenada_UTM_Y', 'Longitud', 'Latitud'])

# we will store the cleaned data in a csv file in order to be used in other Notebooks.
acc_clean.to_csv('../datasets/acc_clean.csv')

## Accidents by hour of the day

In [7]:
number_acc_hour_typeday = acc_clean.groupby(['Type_day', 'Hora_dia'])['Numero_expedient'].count()
acc_hour_typeday = acc_clean.groupby(['Type_day', 'Hora_dia']).aggregate({'Numero_expedient': 'count', 'Numero_victimes': 'sum', 'Numero_vehicles_implicats': 'sum', 'Numero_lesionats_lleus':'sum', 'Numero_lesionats_greus':'sum', 'Numero_morts':'sum'})
acc_hour_typeday.columns = ['Accidents', 'Total_Victims', 'Total_Vehicles', 'Minor_Injuries', 'Severe_Injuries', 'Deaths']
acc_hour_typeday.index.names = ['Type_Day', 'Hour_Day']


acc_hour_typeday_perc = acc_hour_typeday.groupby(level=0).apply(lambda x: round (100 * x / x.sum(),2))
acc_hour_typeday_perc.columns = ['%Accidents', '%Total_Victims', '%Total_Vehicles', '%Minor_Injuries', '%Severe_Injuries', '%Deaths']


acc_hour_typeday = pd.concat([acc_hour_typeday,acc_hour_typeday_perc], axis =1)
acc_hour_typeday = acc_hour_typeday[['Accidents', '%Accidents', 'Total_Victims', '%Total_Victims', 'Total_Vehicles', '%Total_Vehicles', 'Minor_Injuries',
       '%Minor_Injuries', 'Severe_Injuries', '%Severe_Injuries', 'Deaths', '%Deaths']]
acc_hour_typeday

Accidents  %Accidents  Total_Victims  \
Type_Day        Hour_Day                                         
Weekday         0                94        1.19            107   
                1                63        0.80             74   
                2                45        0.57             39   
                3                28        0.35             17   
                4                28        0.35             21   
                5                46        0.58             51   
                6                93        1.18            103   
                7               305        3.86            353   
                8               519        6.56            587   
                9               526        6.65            632   
                10              395        4.99            429   
                11              402        5.08            439   
                12              463        5.85            510   
                13              516        6.52            619   
                14              614        7.76            708   
                15              553        6.99            642   
                16              499        6.31            587   
                17              504        6.37            607   
                18              550        6.95            680   
                19              490        6.19            601   
                20              463        5.85            543   
                21              325        4.11            389   
                22              210        2.65            258   
                23              179        2.26            211   
Weekend/Holiday 0                77        3.64             92   
                1                61        2.88             70   
                2                46        2.17             58   
                3                48        2.27             43   
                4                43        2.03             39   
                5                47        2.22             53   
                6                77        3.64             90   
                7                57        2.69             53   
                8                47        2.22             55   
                9                51        2.41             65   
                10               71        3.35             83   
                11              101        4.77            124   
                12              110        5.20            154   
                13              135        6.38            183   
                14              157        7.42            212   
                15              118        5.57            154   
                16              103        4.87            138   
                17              108        5.10            141   
                18              124        5.86            184   
                19              121        5.72            153   
                20              113        5.34            150   
                21              116        5.48            145   
                22              106        5.01            122   
                23               80        3.78             96   

                          %Total_Victims  Total_Vehicles  %Total_Vehicles  \
Type_Day        Hour_Day                                                    
Weekday         0                   1.16             168             1.12   
                1                   0.80             122             0.81   
                2                   0.42              73             0.49   
                3                   0.18              61             0.41   
                4                   0.23              53             0.35   
                5                   0.55              93             0.62   
                6                   1.12             194             1.29   
                7               

### Hours-day sorted by accidents

In [8]:
acc_hour_typeday[['Accidents','%Accidents']].sort_values(['Type_Day','Accidents'], ascending = [True,False])

Accidents  %Accidents
Type_Day        Hour_Day                       
Weekday         14              614        7.76
                15              553        6.99
                18              550        6.95
                9               526        6.65
                8               519        6.56
                13              516        6.52
                17              504        6.37
                16              499        6.31
                19              490        6.19
                12              463        5.85
                20              463        5.85
                11              402        5.08
                10              395        4.99
                21              325        4.11
                7               305        3.86
                22              210        2.65
                23              179        2.26
                0                94        1.19
                6                93        1.18
                1                63        0.80
                5                46        0.58
                2                45        0.57
                3                28        0.35
                4                28        0.35
Weekend/Holiday 14              157        7.42
                13              135        6.38
                18              124        5.86
                19              121        5.72
                15              118        5.57
                21              116        5.48
                20              113        5.34
                12              110        5.20
                17              108        5.10
                22              106        5.01
                16              103        4.87
                11              101        4.77
                23               80        3.78
                0                77        3.64
                6                77        3.64
                10               71        3.35
                1                61        2.88
                7                57        2.69
                9                51        2.41
                3                48        2.27
                5                47        2.22
                8                47        2.22
                2                46        2.17
                4                43        2.03

### Hours-day sorted by victims

In [9]:
acc_hour_typeday[['Total_Victims','%Total_Victims']].sort_values(['Type_Day','Total_Victims'], ascending = [True,False])

Total_Victims  %Total_Victims
Type_Day        Hour_Day                               
Weekday         14                  708            7.69
                18                  680            7.39
                15                  642            6.97
                9                   632            6.86
                13                  619            6.72
                17                  607            6.59
                19                  601            6.53
                8                   587            6.38
                16                  587            6.38
                20                  543            5.90
                12                  510            5.54
                11                  439            4.77
                10                  429            4.66
                21                  389            4.23
                7                   353            3.83
                22                  258            2.80
                23                  211            2.29
                0                   107            1.16
                6                   103            1.12
                1                    74            0.80
                5                    51            0.55
                2                    39            0.42
                4                    21            0.23
                3                    17            0.18
Weekend/Holiday 14                  212            7.98
                18                  184            6.93
                13                  183            6.89
                12                  154            5.80
                15                  154            5.80
                19                  153            5.76
                20                  150            5.65
                21                  145            5.46
                17                  141            5.31
                16                  138            5.19
                11                  124            4.67
                22                  122            4.59
                23                   96            3.61
                0                    92            3.46
                6                    90            3.39
                10                   83            3.12
                1                    70            2.63
                9                    65            2.45
                2                    58            2.18
                8                    55            2.07
                5                    53            1.99
                7                    53            1.99
                3                    43            1.62
                4                    39            1.47

### Hours-day sorted by severe injuries

In [10]:
acc_hour_typeday[['Severe_Injuries','%Severe_Injuries']].sort_values(['Type_Day','Severe_Injuries'], ascending = [True,False])

Severe_Injuries  %Severe_Injuries
Type_Day        Hour_Day                                   
Weekday         18                     14             10.37
                15                     12              8.89
                19                     12              8.89
                13                     10              7.41
                22                      8              5.93
                14                      7              5.19
                17                      7              5.19
                20                      7              5.19
                21                      7              5.19
                8                       6              4.44
                10                      6              4.44
                11                      6              4.44
                16                      6              4.44
                23                      6              4.44
                1                       5              3.70
                6                       4              2.96
                12                      4              2.96
                0                       3              2.22
                7                       2              1.48
                9                       2              1.48
                2                       1              0.74
                3                       0              0.00
                4                       0              0.00
                5                       0              0.00
Weekend/Holiday 18                     10             14.93
                17                      6              8.96
                21                      5              7.46
                5                       4              5.97
                12                      4              5.97
                19                      4              5.97
                22                      4              5.97
                1                       3              4.48
                2                       3              4.48
                13                      3              4.48
                14                      3              4.48
                20                      3              4.48
                0                       2              2.99
                7                       2              2.99
                8                       2              2.99
                10                      2              2.99
                15                      2              2.99
                3                       1              1.49
                4                       1              1.49
                6                       1              1.49
                16                      1              1.49
                23                      1              1.49
                9                       0              0.00
                11                      0              0.00

### Hours-day sorted by deaths

In [11]:
acc_hour_typeday[['Deaths','%Deaths']].sort_values(['Type_Day','Deaths'], ascending = [True,False])

Deaths  %Deaths
Type_Day        Hour_Day                 
Weekday         16             3    21.43
                8              2    14.29
                3              1     7.14
                4              1     7.14
                7              1     7.14
                12             1     7.14
                13             1     7.14
                14             1     7.14
                18             1     7.14
                19             1     7.14
                21             1     7.14
                0              0     0.00
                1              0     0.00
                2              0     0.00
                5              0     0.00
                6              0     0.00
                9              0     0.00
                10             0     0.00
                11             0     0.00
                15             0     0.00
                17             0     0.00
                20             0     0.00
                22             0     0.00
                23             0     0.00
Weekend/Holiday 6              3    37.50
                5              2    25.00
                4              1    12.50
                13             1    12.50
                22             1    12.50
                0              0     0.00
                1              0     0.00
                2              0     0.00
                3              0     0.00
                7              0     0.00
                8              0     0.00
                9              0     0.00
                10             0     0.00
                11             0     0.00
                12             0     0.00
                14             0     0.00
                15             0     0.00
                16             0     0.00
                17             0     0.00
                18             0     0.00
                19             0     0.00
                20             0     0.00
                21             0     0.00
                23             0     0.00

## Accidents by day of the week

In [12]:
acc_week = acc_clean.groupby('Dia_setmana').aggregate({'Numero_expedient': 'count', 'Numero_victimes': 'sum',
                                                    'Numero_vehicles_implicats': 'sum', 'Numero_lesionats_lleus':'sum',
                                                    'Numero_lesionats_greus':'sum', 'Numero_morts':'sum'})
acc_week.columns = ['Accidents', 'Total_Victims', 'Total_Vehicles', 'Minor_Injuries', 'Severe_Injuries', 'Deaths']
acc_week.index.name = 'Day_week'
acc_week.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
acc_week_perc = acc_week.apply(lambda x: round (100 * x / x.sum(),2))
acc_week_perc.columns = ['%Accidents', '%Total_Victims', '%Total_Vehicles', 
                          '%Minor_Injuries', '%Severe_Injuries', '%Deaths']

acc_week = pd.concat([acc_week, acc_week_perc], axis =1)
acc_week = acc_week[['Accidents', '%Accidents', 'Total_Victims', 
                       '%Total_Victims', 'Total_Vehicles', '%Total_Vehicles', 'Minor_Injuries', 
                       '%Minor_Injuries', 'Severe_Injuries', '%Severe_Injuries', 'Deaths', '%Deaths']]
acc_week

,Accidents,%Accidents,Total_Victims,%Total_Victims,Total_Vehicles,%Total_Vehicles,Minor_Injuries,%Minor_Injuries,Severe_Injuries,%Severe_Injuries,Deaths,%Deaths
Monday,1680,16.75,1947,16.41,3155,16.57,1913,16.43,30,14.85,4,18.18
Tuesday,834,8.32,1040,8.77,1580,8.30,1015,8.72,20,9.90,5,22.73
Wednesday,1648,16.44,1972,16.62,3123,16.40,1936,16.63,32,15.84,4,18.18
Thursday,1505,15.01,1754,14.78,2847,14.96,1733,14.89,18,8.91,3,13.64
Friday,1612,16.08,1895,15.97,3101,16.29,1861,15.99,34,16.83,0,0.00
Saturday,1087,10.84,1367,11.52,2067,10.86,1322,11.36,43,21.29,2,9.09
Sunday,1661,16.57,1889,15.92,3164,16.62,1860,15.98,25,12.38,4,18.18


### Days of the week sorted by accidents

In [13]:
acc_week[['Accidents','%Accidents']].sort_values('Accidents', ascending = False)

,Accidents,%Accidents
Monday,1680,16.75
Sunday,1661,16.57
Wednesday,1648,16.44
Friday,1612,16.08
Thursday,1505,15.01
Saturday,1087,10.84
Tuesday,834,8.32


### Days of the week sorted by victims

In [14]:
acc_week[['Total_Victims','%Total_Victims']].sort_values('Total_Victims', ascending = False)

,Total_Victims,%Total_Victims
Wednesday,1972,16.62
Monday,1947,16.41
Friday,1895,15.97
Sunday,1889,15.92
Thursday,1754,14.78
Saturday,1367,11.52
Tuesday,1040,8.77


### Days of the week sorted by severe injuries

In [15]:
acc_week[['Severe_Injuries','%Severe_Injuries']].sort_values('Severe_Injuries', ascending = False)

,Severe_Injuries,%Severe_Injuries
Saturday,43,21.29
Friday,34,16.83
Wednesday,32,15.84
Monday,30,14.85
Sunday,25,12.38
Tuesday,20,9.90
Thursday,18,8.91


### Days of the week sorted by deaths

In [16]:
acc_week[['Deaths','%Deaths']].sort_values('Deaths', ascending = False)

,Deaths,%Deaths
Tuesday,5,22.73
Monday,4,18.18
Wednesday,4,18.18
Sunday,4,18.18
Thursday,3,13.64
Saturday,2,9.09
Friday,0,0.00


## Accidents by month of the year

In [17]:
acc_month = acc_clean.groupby('Mes_any').aggregate({'Numero_expedient': 'count', 'Numero_victimes': 'sum',
                                                    'Numero_vehicles_implicats': 'sum', 'Numero_lesionats_lleus':'sum',
                                                    'Numero_lesionats_greus':'sum', 'Numero_morts':'sum'})
acc_month.columns = ['Accidents', 'Total_Victims', 'Total_Vehicles', 'Minor_Injuries', 'Severe_Injuries', 'Deaths']
acc_month.index.name = 'Month'
acc_month['Month_Name'] = pd.Series(['','January', 'February', 'March', 'April', 'May', 'June', 
                                     'July', 'August', 'September', 'October', 'November', 'December'])
acc_month = acc_month[['Month_Name','Accidents', 'Total_Victims', 'Total_Vehicles', 
                       'Minor_Injuries', 'Severe_Injuries', 'Deaths'] ]

acc_month_perc = acc_month[['Accidents', 'Total_Victims', 'Total_Vehicles', 'Minor_Injuries', 
                           'Severe_Injuries', 'Deaths']].apply(lambda x: round (100 * x / x.sum(),2))
acc_month_perc.columns = ['%Accidents', '%Total_Victims', '%Total_Vehicles', 
                          '%Minor_Injuries', '%Severe_Injuries', '%Deaths']


acc_month = pd.concat([acc_month, acc_month_perc], axis =1)
acc_month = acc_month[['Month_Name','Accidents', '%Accidents', 'Total_Victims', '%Total_Victims', 'Total_Vehicles', '%Total_Vehicles', 'Minor_Injuries', '%Minor_Injuries', 'Severe_Injuries', '%Severe_Injuries', 'Deaths', '%Deaths']]
acc_month

,Month_Name,Accidents,%Accidents,Total_Victims,%Total_Victims,Total_Vehicles,%Total_Vehicles,Minor_Injuries,%Minor_Injuries,Severe_Injuries,%Severe_Injuries,Deaths,%Deaths
Month,,,,,,,,,,,,,
1,January,845,8.43,965,8.13,1600,8.40,944,8.11,19,9.41,2,9.09
2,February,781,7.79,919,7.75,1564,8.22,903,7.76,15,7.43,1,4.55
3,March,885,8.83,1063,8.96,1695,8.90,1046,8.99,16,7.92,1,4.55
4,April,813,8.11,988,8.33,1570,8.25,968,8.32,18,8.91,2,9.09
5,May,944,9.41,1111,9.36,1775,9.32,1094,9.40,16,7.92,1,4.55
6,June,886,8.84,1064,8.97,1687,8.86,1044,8.97,19,9.41,1,4.55
7,July,947,9.44,1124,9.47,1800,9.46,1098,9.43,22,10.89,4,18.18
8,August,655,6.53,798,6.73,1217,6.39,784,6.74,13,6.44,1,4.55
9,September,753,7.51,912,7.69,1409,7.40,895,7.69,16,7.92,1,4.55


### Months sorted by accidents

In [18]:
acc_month[['Month_Name','Accidents','%Accidents']].sort_values('Accidents', ascending = False)

,Month_Name,Accidents,%Accidents
Month,,,
7,July,947,9.44
5,May,944,9.41
10,October,917,9.15
6,June,886,8.84
3,March,885,8.83
11,November,846,8.44
1,January,845,8.43
4,April,813,8.11
2,February,781,7.79


### Months sorted by victims

In [19]:
acc_month[['Month_Name','Total_Victims','%Total_Victims']].sort_values('Total_Victims', ascending = False)

,Month_Name,Total_Victims,%Total_Victims
Month,,,
7,July,1124,9.47
5,May,1111,9.36
6,June,1064,8.97
3,March,1063,8.96
10,October,1057,8.91
4,April,988,8.33
1,January,965,8.13
11,November,963,8.12
2,February,919,7.75


### Months sorted by severe injuries

In [20]:
acc_month[['Month_Name','Severe_Injuries','%Severe_Injuries']].sort_values('Severe_Injuries', ascending = False)

,Month_Name,Severe_Injuries,%Severe_Injuries
Month,,,
7,July,22,10.89
10,October,21,10.40
1,January,19,9.41
6,June,19,9.41
4,April,18,8.91
3,March,16,7.92
5,May,16,7.92
9,September,16,7.92
2,February,15,7.43


### Months sorted by deaths

In [21]:
acc_month[['Month_Name','Deaths','%Deaths']].sort_values('Deaths', ascending = False)

,Month_Name,Deaths,%Deaths
Month,,,
7,July,4,18.18
10,October,3,13.64
11,November,3,13.64
1,January,2,9.09
4,April,2,9.09
12,December,2,9.09
2,February,1,4.55
3,March,1,4.55
5,May,1,4.55
